---
title: "Data Cleaning"
format:
  html:
    toc: true
---

In this section, we will walk through the data cleaning process for my wildfire prediction and mitigation project. The primary goal is to prepare the raw data for analysis and modeling. I used R code to clean record data that has both qualitative and quantitative variables, and python code to clean labeled text data. To access the original data cleaning codes, please visit the Github link here [^1].

[^1]: My Data Cleaning Codes - [Link](https://github.com/anly501/dsan-5000-project-jtjt427/tree/main/codes/02-data-cleaning)

## Wildfire Statistics

### Data Loading

We start by loading the raw data from CSV files, including HumanCausedAcres.csv, HumanCausedFires.csv, LighteningAcres.csv, and LightningFires.csv. These files contain various data related to wildfires statistics across the states.

Note that the four raw datasets share an extremely similiar data structure, except that HumanCausedAcres.csv and LighteningAcres.csv are the statistics on wildfire **acres** caused by human and lightening, whereas HumanCausedFires.csv and LightningFires.csv are the statistics on the **number** of wildfires caused by human and lightening.(Details on the raw datasets are demonstrated under the Data Cleaning tab). Therefore, I will use HumanCausedAcres.csv as an example to illustrate on the data cleaning process, and the rest of the three datasets follow the exact same cleaning goal and process.

### Subset Columns

In [37]:
#| echo: false
library(readr)
library(dplyr)
library(ggplot2)
library(tidyr)
HumanCausedAcres <- read_csv("../data/raw-data/HumanCausedAcres.csv", show_col_types = FALSE)

In [25]:
head(HumanCausedAcres)

Year,Alaska,Northwest,Northern California,Southern California,Northern Rockies,Great Basin,Western Great Basin,Southwest,Rocky Mountains,Eastern Area,Southern Area,Total
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
2020,180885,4667060,1549012,216492,172118,517325,N/A,671504,229995,3178,115994,4123523
2019,44061,140321,190426,36416,31984,126584,NA,202918,49743,38773,356188,1217324
2018,28946,609578,1404463,324102,48372,1030761,N/A,309546,504416,374,1329945,5640489
2017,6890,465864,343195,532640,135131,788769,N/A,207818,645162,41459,1663548,4830476
2016,10069,436106,93699,464718,84249,333318,N/A,219103,530831,97094,1497423,3766610
2015,26652,702206,308762,96990,77905,49066,N/A,19596,163871,100094,467319,2012461


**Column Selection**: Initially, we have all columns from the raw data. However, for our analysis, we need a subset of columns that are relevant. These typically include `Year`, `Northern California`, `Southern California`, and `Total`. The columns are chosen to focus on the analysis on the trend of wildfire across the states:

In [26]:
HumanCausedAcres <- HumanCausedAcres[, c("Year", "Northern California", 
                                         "Southern California", "Total")]
head(HumanCausedAcres)

Year,Northern California,Southern California,Total
<dbl>,<dbl>,<dbl>,<dbl>
2020,1549012,216492,4123523
2019,190426,36416,1217324
2018,1404463,324102,5640489
2017,343195,532640,4830476
2016,93699,464718,3766610
2015,308762,96990,2012461


### Create New Column and Remove Unnecessary Columns

**Consolidating Acreage**: To simplify the representation of acreage data, we create a new column called `California_Acres` in the `HumanCausedAcres` data frame. This new column represents the combined acreage for both Northern and Southern California.

**Streamlining Data**: After creating the `California_Acres` column, we remove the `Northern California` and `Southern California` columns from the data frame since they are no longer needed for our analysis:

In [27]:
HumanCausedAcres$California_Acres <- HumanCausedAcres$`Northern California` +
  HumanCausedAcres$`Southern California`
# Remove the column Northern California` and column `Southern California`
HumanCausedAcres <- subset(HumanCausedAcres, select = -c(`Northern California`, 
                                                         `Southern California`))
head(HumanCausedAcres)

Year,Total,California_Acres
<dbl>,<dbl>,<dbl>
2020,4123523,1765504
2019,1217324,226842
2018,5640489,1728565
2017,4830476,875835
2016,3766610,558417
2015,2012461,405752


### Rename Columns

**Clarity and Consistency**: To ensure clarity and consistency in column naming, I rename the 'Total' column to 'Total_Acres'. This change accurately reflects the content of the column, which avoids the mergeing process:

In [28]:
HumanCausedAcres <- HumanCausedAcres %>%
  rename(Total_Acres = Total)
# Switch the order of the two columns
HumanCausedAcres <- HumanCausedAcres[, c("Year", "California_Acres", "Total_Acres")]
head(HumanCausedAcres)

Year,California_Acres,Total_Acres
<dbl>,<dbl>,<dbl>
2020,1765504,4123523
2019,226842,1217324
2018,1728565,5640489
2017,875835,4830476
2016,558417,3766610
2015,405752,2012461


Repeating the above cleaning procedures on the rest of three datasets and get four cleaned up wildfire datasets for this study:

In [29]:
#| code-fold: true
# Do the same for the rest of the data frames
HumanCausedFires <- read_csv("../data/raw-data/HumanCausedFires.csv", show_col_types = FALSE)
LighteningAcres <- read_csv("../data/raw-data/LighteningAcres.csv", show_col_types = FALSE)
LightingFires <- read_csv("../data/raw-data/LightingFires.csv", show_col_types = FALSE)
HumanCausedFires <- HumanCausedFires[, c("Year", "Northern California",
                                         "Southern California", "Total")]
LighteningAcres <- LighteningAcres[, c("Year", "Northern California",
                                       "Southern California", "Total")]
LightingFires <- LightingFires[, c("Year", "Northern California",
                                   "Southern California", "Total")]

HumanCausedFires$California_Fires <- HumanCausedFires$`Northern California` +
  HumanCausedFires$`Southern California`
HumanCausedFires <- subset(HumanCausedFires, select = -c(`Northern California`, 
                                                         `Southern California`))
HumanCausedFires <- HumanCausedFires %>%
  rename(Total_Fires = Total)
HumanCausedFires <- HumanCausedFires[, c("Year", "California_Fires", "Total_Fires")]

LighteningAcres$California_Acres <- LighteningAcres$`Northern California` +
  LighteningAcres$`Southern California`
LighteningAcres <- subset(LighteningAcres, select = -c(`Northern California`, 
                                                       `Southern California`))
LighteningAcres <- LighteningAcres %>%
  rename(Total_Acres = Total)
LighteningAcres <- LighteningAcres[, c("Year", "California_Acres", "Total_Acres")]

LightingFires$California_Fires <- LightingFires$`Northern California` +
  LightingFires$`Southern California`
LightingFires <- subset(LightingFires, select = -c(`Northern California`, 
                                                   `Southern California`))
LightingFires <- LightingFires %>%
  rename(Total_Fires = Total)
LightingFires <- LightingFires[, c("Year", "California_Fires", "Total_Fires")]

### Merge Data Frames

**Data Integration**: We merge two data frames, 'HumanCausedAcres' and 'HumanCausedFires', using the `Year` column as the common identifier. Similarly, two other data frames, 'LighteningAcres' and 'LightingFires,' are also merged based on the `Year` column. The two merged datasets are called 'HumanCaused' and 'LighteningCaused' respectively:

In [30]:
HumanCaused <- merge(HumanCausedAcres, HumanCausedFires, by = "Year", all = TRUE)
LighteningCaused <- merge(LighteningAcres, LightingFires, by = "Year", all = TRUE)

head(HumanCaused)
head(LighteningCaused)

,Year,California_Acres,Total_Acres,California_Fires,Total_Fires
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,2001,202480,1748661,5159,70066
2,2002,452007,3077119,7849,62022
3,2003,749431,1922249,7724,50815
4,2004,230795,964800,7458,54101
5,2005,99386,1521327,6791,58430
6,2006,489863,4404844,6842,80220


,Year,California_Acres,Total_Acres,California_Fires,Total_Fires
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,2001,320901,1822600,3070,14094
2,2002,58349,4097593,480,11435
3,2003,50436,2038443,1394,12776
4,2004,12022,7011023,993,11384
5,2005,104867,7168062,458,8323
6,2006,198886,5468901,1357,16165


We observe that the two datasets have the same column names. Because my goal is to conbine the datasets into one big chart, we need to rename the columns to avoid confusion:

In [31]:
# Rename the columns
HumanCaused <- HumanCaused %>%
  rename(Cali_Acres_H = California_Acres,
         Total_Acres_H = Total_Acres, 
         Cali_Fires_H = California_Fires,
         Total_Fires_H = Total_Fires)

LighteningCaused <- LighteningCaused %>%
  rename(Cali_Acres_L = California_Acres,
         Total_Acres_L = Total_Acres, 
         Cali_Fires_L = California_Fires,
         Total_Fires_L = Total_Fires)

head(HumanCaused)
head(LighteningCaused)

,Year,Cali_Acres_H,Total_Acres_H,Cali_Fires_H,Total_Fires_H
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,2001,202480,1748661,5159,70066
2,2002,452007,3077119,7849,62022
3,2003,749431,1922249,7724,50815
4,2004,230795,964800,7458,54101
5,2005,99386,1521327,6791,58430
6,2006,489863,4404844,6842,80220


,Year,Cali_Acres_L,Total_Acres_L,Cali_Fires_L,Total_Fires_L
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,2001,320901,1822600,3070,14094
2,2002,58349,4097593,480,11435
3,2003,50436,2038443,1394,12776
4,2004,12022,7011023,993,11384
5,2005,104867,7168062,458,8323
6,2006,198886,5468901,1357,16165


Now, I am safe to merge the datasets:

In [32]:
Human_Lightening <- merge(HumanCaused, LighteningCaused, by = "Year", all = TRUE)
head(Human_Lightening)

,Year,Cali_Acres_H,Total_Acres_H,Cali_Fires_H,Total_Fires_H,Cali_Acres_L,Total_Acres_L,Cali_Fires_L,Total_Fires_L
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,2001,202480,1748661,5159,70066,320901,1822600,3070,14094
2,2002,452007,3077119,7849,62022,58349,4097593,480,11435
3,2003,749431,1922249,7724,50815,50436,2038443,1394,12776
4,2004,230795,964800,7458,54101,12022,7011023,993,11384
5,2005,99386,1521327,6791,58430,104867,7168062,458,8323
6,2006,489863,4404844,6842,80220,198886,5468901,1357,16165


Since the column names are still variables, we make a new column called cause that stores the cause of the wildfire:

In [33]:
Human_Lightening <- Human_Lightening %>%
  pivot_longer(cols = -Year, names_to = "Variable") %>%
  mutate(
    Cause = ifelse(grepl("_H$", Variable), "Human", "Lightning"),
    Variable = gsub("_H$|_L$", "", Variable)
  ) %>%
  pivot_wider(names_from = Variable, values_from = value)

head(Human_Lightening)

Year,Cause,Cali_Acres,Total_Acres,Cali_Fires,Total_Fires
<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>
2001,Human,202480,1748661,5159,70066
2001,Lightning,320901,1822600,3070,14094
2002,Human,452007,3077119,7849,62022
2002,Lightning,58349,4097593,480,11435
2003,Human,749431,1922249,7724,50815
2003,Lightning,50436,2038443,1394,12776


## Suppression Cost Statistics

In this section, we will discuss on the data cleaning process for the 'SuppCosts' dataset, which contains information about wildfire suppression costs. we will clean, transform, and organize the data to prepare it for analysis.

### Data Loading

we begin by loading the raw "SuppCosts" dataset from the CSV file. This dataset contains columns such as `Year`, `Fires`, `Acres`, `Forest Service`, `DOI Agencies` and `Total`, which provide information about suppression costs over the years:

In [42]:
SuppCosts <- read_csv("../data/raw-data/SuppCosts.csv", show_col_types = FALSE)
head(SuppCosts)

Year,Fires,Acres,Forest Service,DOI Agencies,Total
<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>
2022,68988,"7,577,183","$2,900,000,000","$649,000,000","$3,549,000,000"
2021,58985,"7,125,643","$3,741,000,000","$648,000,000","$4,389,000,000"
2020,58950,"10,122,336","$1,764,000,000","$510,000,000","$2,274,000,000"
2019,50477,"4,664,364","$1,150,000,000","$440,000,000","$1,590,000,000"
2018,58083,"8,767,492","$2,615,256,000","$528,000,000","$3,143,256,000"
2017,71499,"10,026,086","$2,410,165,000","$508,000,000","$2,918,165,000"


### Subset Columns

**Column Selection**: Initially, we have all the columns from the raw data. However, for our analysis, we need a subset of columns that are relevant. we select the columns `Year`, `Fires`, `Acres`, and `Total` to focus on wildfire suppression cost trends:

In [43]:
SuppCosts <- SuppCosts[, c("Year", "Fires", "Acres", "Total")]
head(SuppCosts)

Year,Fires,Acres,Total
<dbl>,<dbl>,<chr>,<chr>
2022,68988,"7,577,183","$3,549,000,000"
2021,58985,"7,125,643","$4,389,000,000"
2020,58950,"10,122,336","$2,274,000,000"
2019,50477,"4,664,364","$1,590,000,000"
2018,58083,"8,767,492","$3,143,256,000"
2017,71499,"10,026,086","$2,918,165,000"


### Remove Rows and Convert Columns

**Data Cleanup**: 
- we remove rows below index 38 from the dataset since it is a N/A row.
- we convert the "Acres" and "Total" columns to numeric format by removing commas and dollar signs:

In [44]:
SuppCosts <- SuppCosts[(1:38), ]

SuppCosts$Acres <- as.numeric(gsub(",", "", SuppCosts$Acres))
SuppCosts$Total <- as.numeric(gsub("\\$", "", gsub(",", "", SuppCosts$Total)))

### Rename Columns

**Clarity and Consistency**: To ensure clear and consistent column naming, we rename the `Total` column to `Total_spent`. This change accurately reflects the content of the column:

In [45]:
SuppCosts <- SuppCosts  %>%
  rename(Total_spent = Total)
head(SuppCosts)

Year,Fires,Acres,Total_spent
<dbl>,<dbl>,<dbl>,<dbl>
2022,68988,7577183,3549000000
2021,58985,7125643,4389000000
2020,58950,10122336,2274000000
2019,50477,4664364,1590000000
2018,58083,8767492,3143256000
2017,71499,10026086,2918165000


### Calculate Combined Score

**Data Enrichment**: we calculate a new column called `CombinedScore` by summing the `Fires`and `Acres` columns. This combined score represents the overall impact of wildfires, taking both the number of fires and acres affected into account.

In [46]:
# Calculate a combined score based on Fires and Acres
SuppCosts$CombinedScore <- SuppCosts$Fires + SuppCosts$Acres
head(SuppCosts)

Year,Fires,Acres,Total_spent,CombinedScore
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
2022,68988,7577183,3549000000,7646171
2021,58985,7125643,4389000000,7184628
2020,58950,10122336,2274000000,10181286
2019,50477,4664364,1590000000,4714841
2018,58083,8767492,3143256000,8825575
2017,71499,10026086,2918165000,10097585


### Sort Data

**Data Analysis**: To facilitate analysis, we sort the data based on the `CombinedScore` column in descending order. This arrangement allows me to identify the most significant wildfire events.

In [47]:
# Sort the data based on the combined score in descending order
sorted_SuppCosts <- SuppCosts[order(-SuppCosts$CombinedScore), ]

In [48]:
head(sorted_SuppCosts)

Year,Fires,Acres,Total_spent,CombinedScore
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
2015,68151,10125149,2130543000,10193300
2020,58950,10122336,2274000000,10181286
2017,71499,10026086,2918165000,10097585
2006,96385,9873745,1704477000,9970130
2007,85705,9328045,1620145000,9413750
2012,67774,9326238,1902446000,9394012


## California Wildfire Statistics

In this section, we will walk through the data cleaning process for the `California wildfire statistics` datasets, which contains information on year-to-date wildfires in California in the year of 2020, 2017, 2015, 2014, 2010. The goal is to prepare this dataset for analysis and modeling on the relationship between weather and wildfire, ensuring that the data is in a cleaner and more usable state.

### Data Loading

We start by loading the raw "Cali_20_Wildfire" dataset from a CSV file. The dataset initially includes various columns, such as `StartDate`, `Acres Burned`, `Cause`, `StructureDest.`, `StructureDam.`, `FirePersonnelDeath`, and `CivilDeath`:

In [38]:
Cali_20_Wildfire <- read_csv("../data/raw-data/Cali_20_Wildfire.csv", show_col_types = FALSE)
head(Cali_20_Wildfire)

Incident #,County,FireName,StartDate,EndDate,Acres Burned,VegType,Cause,StructureDest.,StructureDam.,FirePersonnelDeath,CivilDeath
<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>
NEU-11799,PLACER,AMORUSO,6/1/20,6/1/20,670,G,EQUIPMENT USE,NA,NA,NA,NA
LNU-8419,SOLANO,QUAIL,6/6/20,6/13/20,1837,T,FIREARMS,5,12,NA,NA
SLU-7149,SAN LUIS OBISPO,SODA,6/10/20,6/10/20,1680,G,VEHICLE,NA,NA,NA,NA
VNC-45530,VENTURA,ELIZABETH,6/10/20,6/12/20,300,B,UNDETERMINED,NA,NA,NA,NA
VNC-45754,VENTURA,LIME,6/10/20,6/16/20,803,"B, G",UNDETERMINED,NA,NA,NA,NA
AEU-16543,SACRAMENTO,GRANT,6/12/20,6/19/20,5042,G,EQUIPMENT USE,NA,1,NA,NA


### Subset Columns

**Column Selection**: To focus on the analysis and modeling, we select a subset of relevant columns, including `StartDate`, `AcresBurned`, `Cause`, `StructureDest.`, `StructureDam.`, `FirePersonnelDeath`, and `CivilDeath`. These columns are chosen based on the nature of the analysis:

In [39]:
Cali_20_Wildfire <- Cali_20_Wildfire %>%
  rename(
    AcresBurned = `Acres Burned`
  )
Cali_20_Wildfire <- Cali_20_Wildfire[ , c('StartDate', 'AcresBurned', 'Cause', 
                          'StructureDest.', 'StructureDam.', 
                          'FirePersonnelDeath', 'CivilDeath')]
head(Cali_20_Wildfire)

StartDate,AcresBurned,Cause,StructureDest.,StructureDam.,FirePersonnelDeath,CivilDeath
<chr>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>
6/1/20,670,EQUIPMENT USE,NA,NA,NA,NA
6/6/20,1837,FIREARMS,5,12,NA,NA
6/10/20,1680,VEHICLE,NA,NA,NA,NA
6/10/20,300,UNDETERMINED,NA,NA,NA,NA
6/10/20,803,UNDETERMINED,NA,NA,NA,NA
6/12/20,5042,EQUIPMENT USE,NA,1,NA,NA


### Replace Missing Values

**Missing Value Management**: we address missing values in the dataset by replacing them with zeros. Specifically, we replace NA values in columns "StructureDest.," "StructureDam.," "FirePersonnelDeath," and "CivilDeath" with zeros. The purpose of this step is to prepare the dataset for the following column merging steps:

In [40]:
Cali_20_Wildfire$StructureDest.[is.na(Cali_20_Wildfire$StructureDest.)] <- 0
Cali_20_Wildfire$StructureDam.[is.na(Cali_20_Wildfire$StructureDam.)] <- 0
Cali_20_Wildfire$FirePersonnelDeath[is.na(Cali_20_Wildfire$FirePersonnelDeath)] <- 0
Cali_20_Wildfire$CivilDeath[is.na(Cali_20_Wildfire$CivilDeath)] <- 0

### Create New Columns

**Feature Engineering**: we create two new columns in the dataset:

1. `StructureDam`: This column is created by adding `StructureDest.` and `StructureDam`. This represents the combined impact on structures from wildfires.

2. `Fatalities`: This column is created by adding `FirePersonnelDeath` and `CivilDeath`. It represents the total number of fatalities from wildfires.

### Remove Original Columns

**Data Reduction**: We remove the original columns `StructureDest.`, `StructureDam.`, `FirePersonnelDeath`, and `CivilDeath` from the dataset as they are no longer needed for the analysis:

In [41]:
# Add up 'StructureDest.' and 'StructureDam.' and name the new column 'StructureDam'
Cali_20_Wildfire$StructureDam <- Cali_20_Wildfire$StructureDest. + Cali_20_Wildfire$StructureDam.

# Add up 'FirePersonnelDeath' and 'CivilDeath' and name the new column 'Fatalities'
Cali_20_Wildfire$Fatalities <- Cali_20_Wildfire$FirePersonnelDeath + Cali_20_Wildfire$CivilDeath

# Delete the original four columns
Cali_20_Wildfire <- Cali_20_Wildfire[, !names(Cali_20_Wildfire) %in% c('StructureDest.', 'StructureDam.', 'FirePersonnelDeath', 'CivilDeath')]


In [9]:
head(Cali_20_Wildfire)

StartDate,AcresBurned,Cause,StructureDam,Fatalities
<chr>,<dbl>,<chr>,<dbl>,<dbl>
6/1/20,670,EQUIPMENT USE,0,0
6/6/20,1837,FIREARMS,17,0
6/10/20,1680,VEHICLE,0,0
6/10/20,300,UNDETERMINED,0,0
6/10/20,803,UNDETERMINED,0,0
6/12/20,5042,EQUIPMENT USE,1,0



### Date Column Cleanup

**Date Format Conversion**: we convert the `StartDate` column to Date format, using the format "%m/%d/%y". This ensures that the date values are consistent and usable for future data merging.

**Data Integrity**: Rows with missing values in the `StartDate` column are identified and removed from the dataset, ensuring that the dataset contains only records with valid date information:


In [42]:
# Convert date column to Date format
Cali_20_Wildfire$StartDate <- as.Date(Cali_20_Wildfire$StartDate, format = "%m/%d/%y")
# Identify NA values in the StartDate column
na_rows <- is.na(Cali_20_Wildfire$StartDate)
# Remove rows with NA values in the StartDate column
Cali_20_Wildfire <- Cali_20_Wildfire[!na_rows, ]

In [11]:
head(Cali_20_Wildfire)

StartDate,AcresBurned,Cause,StructureDam,Fatalities
<date>,<dbl>,<chr>,<dbl>,<dbl>
2020-06-01,670,EQUIPMENT USE,0,0
2020-06-06,1837,FIREARMS,17,0
2020-06-10,1680,VEHICLE,0,0
2020-06-10,300,UNDETERMINED,0,0
2020-06-10,803,UNDETERMINED,0,0
2020-06-12,5042,EQUIPMENT USE,1,0


With the cleaning process complete, the "Cali_20_Wildfire" dataset is now in a cleaner state and ready for further analysis and modeling.

Later, we follow the same data cleaning process to clean up the rest of the four datasets (California wildfire statistics in year 2017, 2015, 2014 and 2010).

In [43]:
#| code-fold: true
Cali_17_Wildfire <- read_csv("../data/raw-data/Cali_17_Wildfire.csv", show_col_types = FALSE)
Cali_17_Wildfire <- Cali_17_Wildfire %>%
  rename(
    AcresBurned = `Acres Burned`
  )
Cali_17_Wildfire <- Cali_17_Wildfire[ , c('StartDate', 'AcresBurned', 'Cause', 
                                          'StructureDest.', 'StructureDam.', 
                                          'FirePersonnelDeath', 'CivilDeath')]

# Replace NA values with 0 in the columns
Cali_17_Wildfire$StructureDest.[is.na(Cali_17_Wildfire$StructureDest.)] <- 0
Cali_17_Wildfire$StructureDam.[is.na(Cali_17_Wildfire$StructureDam.)] <- 0
Cali_17_Wildfire$FirePersonnelDeath[is.na(Cali_17_Wildfire$FirePersonnelDeath)] <- 0
Cali_17_Wildfire$CivilDeath[is.na(Cali_17_Wildfire$CivilDeath)] <- 0

# Add up 'StructureDest.' and 'StructureDam.' and name the new column 'StructureDam'
Cali_17_Wildfire$StructureDam <- Cali_17_Wildfire$StructureDest. + Cali_17_Wildfire$StructureDam.

# Add up 'FirePersonnelDeath' and 'CivilDeath' and name the new column 'Fatalities'
Cali_17_Wildfire$Fatalities <- Cali_17_Wildfire$FirePersonnelDeath + Cali_17_Wildfire$CivilDeath

# Delete the original four columns
Cali_17_Wildfire <- Cali_17_Wildfire[, !names(Cali_17_Wildfire) %in% c('StructureDest.', 'StructureDam.', 'FirePersonnelDeath', 'CivilDeath')]

# Identify NA values in the StartDate column
na_rows <- is.na(Cali_17_Wildfire$StartDate)

# Convert date column to Date format
Cali_17_Wildfire$StartDate <- as.Date(Cali_17_Wildfire$StartDate, format = "%m/%d/%y")

# Remove rows with NA values in the StartDate column
Cali_17_Wildfire <- Cali_17_Wildfire[!na_rows, ]


Cali_15_Wildfire <- read_csv("../data/raw-data/Cali_15_Wildfire.csv", show_col_types = FALSE)
Cali_15_Wildfire <- Cali_15_Wildfire %>%
  rename(
    AcresBurned = `Acres Burned`
  )

Cali_15_Wildfire <- Cali_15_Wildfire[ , c('StartDate', 'AcresBurned', 'Cause', 
                                          'StructureDest.', 'StructureDam.', 
                                          'FirePersonnelDeath', 'CivilDeath')]

# Replace NA values with 0 in the columns
Cali_15_Wildfire$StructureDest.[is.na(Cali_15_Wildfire$StructureDest.)] <- 0
Cali_15_Wildfire$StructureDam.[is.na(Cali_15_Wildfire$StructureDam.)] <- 0
Cali_15_Wildfire$FirePersonnelDeath[is.na(Cali_15_Wildfire$FirePersonnelDeath)] <- 0
Cali_15_Wildfire$CivilDeath[is.na(Cali_15_Wildfire$CivilDeath)] <- 0

# Add up 'StructureDest.' and 'StructureDam.' and name the new column 'StructureDam'
Cali_15_Wildfire$StructureDam <- Cali_15_Wildfire$StructureDest. + Cali_15_Wildfire$StructureDam.

# Add up 'FirePersonnelDeath' and 'CivilDeath' and name the new column 'Fatalities'
Cali_15_Wildfire$Fatalities <- Cali_15_Wildfire$FirePersonnelDeath + Cali_15_Wildfire$CivilDeath

# Delete the original four columns
Cali_15_Wildfire <- Cali_15_Wildfire[, !names(Cali_15_Wildfire) %in% c('StructureDest.', 'StructureDam.', 'FirePersonnelDeath', 'CivilDeath')]

# Identify NA values in the StartDate column
na_rows <- is.na(Cali_15_Wildfire$StartDate)

# Convert date column to Date format
Cali_15_Wildfire$StartDate <- as.Date(Cali_15_Wildfire$StartDate, format = "%m/%d/%y")

# Remove rows with NA values in the StartDate column
Cali_15_Wildfire <- Cali_15_Wildfire[!na_rows, ]

Cali_14_Wildfire <- read_csv("../data/raw-data/Cali_14_Wildfire.csv", show_col_types = FALSE)
Cali_14_Wildfire <- Cali_14_Wildfire %>%
  rename(
    AcresBurned = `Acres Burned`
  )
Cali_14_Wildfire <- Cali_14_Wildfire[ , c('StartDate', 'AcresBurned', 'Cause', 
                                          'StructureDest.', 'StructureDam.', 
                                          'FirePersonnelDeath', 'CivilDeath')]

# Replace NA values with 0 in the columns
Cali_14_Wildfire$StructureDest.[is.na(Cali_14_Wildfire$StructureDest.)] <- 0
Cali_14_Wildfire$StructureDam.[is.na(Cali_14_Wildfire$StructureDam.)] <- 0
Cali_14_Wildfire$FirePersonnelDeath[is.na(Cali_14_Wildfire$FirePersonnelDeath)] <- 0
Cali_14_Wildfire$CivilDeath[is.na(Cali_14_Wildfire$CivilDeath)] <- 0

# Add up 'StructureDest.' and 'StructureDam.' and name the new column 'StructureDam'
Cali_14_Wildfire$StructureDam <- Cali_14_Wildfire$StructureDest. + Cali_14_Wildfire$StructureDam.

# Add up 'FirePersonnelDeath' and 'CivilDeath' and name the new column 'Fatalities'
Cali_14_Wildfire$Fatalities <- Cali_14_Wildfire$FirePersonnelDeath + Cali_14_Wildfire$CivilDeath

# Delete the original four columns
Cali_14_Wildfire <- Cali_14_Wildfire[, !names(Cali_14_Wildfire) %in% c('StructureDest.', 'StructureDam.', 'FirePersonnelDeath', 'CivilDeath')]

# Identify NA values in the StartDate column
na_rows <- is.na(Cali_14_Wildfire$StartDate)

# Convert date column to Date format
Cali_14_Wildfire$StartDate <- as.Date(Cali_14_Wildfire$StartDate, format = "%m/%d/%y")

# Remove rows with NA values in the StartDate column
Cali_14_Wildfire <- Cali_14_Wildfire[!na_rows, ]

Cali_10_Wildfire <- read_csv("../data/raw-data/Cali_10_Wildfire.csv", show_col_types = FALSE)
Cali_10_Wildfire <- Cali_10_Wildfire %>%
  rename(
    AcresBurned = `Acres Burned`
  )
Cali_10_Wildfire <- Cali_10_Wildfire[ , c('StartDate', 'AcresBurned', 'Cause', 
                                          'StructureDest.', 'StructureDam.', 
                                          'FirePersonnelDeath', 'CivilDeath')]

# Replace NA values with 0 in the columns
Cali_10_Wildfire$StructureDest.[is.na(Cali_10_Wildfire$StructureDest.)] <- 0
Cali_10_Wildfire$StructureDam.[is.na(Cali_10_Wildfire$StructureDam.)] <- 0
Cali_10_Wildfire$FirePersonnelDeath[is.na(Cali_10_Wildfire$FirePersonnelDeath)] <- 0
Cali_10_Wildfire$CivilDeath[is.na(Cali_10_Wildfire$CivilDeath)] <- 0

# Add up 'StructureDest.' and 'StructureDam.' and name the new column 'StructureDam'
Cali_10_Wildfire$StructureDam <- Cali_10_Wildfire$StructureDest. + Cali_10_Wildfire$StructureDam.

# Add up 'FirePersonnelDeath' and 'CivilDeath' and name the new column 'Fatalities'
Cali_10_Wildfire$Fatalities <- Cali_10_Wildfire$FirePersonnelDeath + Cali_10_Wildfire$CivilDeath

# Delete the original four columns
Cali_10_Wildfire <- Cali_10_Wildfire[, !names(Cali_10_Wildfire) %in% c('StructureDest.', 'StructureDam.', 'FirePersonnelDeath', 'CivilDeath')]

# Identify NA values in the StartDate column
na_rows <- is.na(Cali_10_Wildfire$StartDate)

# Convert date column to Date format
Cali_10_Wildfire$StartDate <- as.Date(Cali_10_Wildfire$StartDate, format = "%m/%d/%y")

# Remove rows with NA values in the StartDate column
Cali_10_Wildfire <- Cali_10_Wildfire[!na_rows, ]

## California Weather Data Statistics

In this section, we will outline the data cleaning process for the `Cali climate` datasets, which include climate data from various years (2020, 2017, 2015, 2014, and 2010). Our goal is to: 
1. prepare these datasets for analysis against the `SuppCosts` dataset on the effects of precipitation on wildfire occurance throughout the years 
2. integrate weather datasets and wildfire datasets with respect to year, in order to facilitate the future analysis on climate effects on wildfires in California.

Since the climate datasets for different years have the same exact data structure, we will use the California climate data in year 2020 to demonstrate the cleaning process.

### Data Loading

We begin by loading multiple raw datasets, each containing climate data for different years. The datasets include information such as `datetime`, `tempmax`, `tempmin`, `humidity`, `precip`, and `windspeed`:


In [44]:
Cali_2020 <- read_csv("../data/raw-data/Cali_2020.csv", show_col_types = FALSE)

In [45]:
head(Cali_2020)

datetime,datetimeEpoch,tempmax,tempmin,temp,feelslikemax,feelslikemin,feelslike,dew,humidity,...,sunset,sunsetEpoch,moonphase,conditions,description,icon,stations,source,tzoffset,events
<date>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,...,<time>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>
2020-01-01,1577865600,14.3,5.4,9.0,14.3,4.7,8.7,6.7,85.7,...,16:55:17,1577926517,0.21,Partially cloudy,Becoming cloudy in the afternoon.,partly-cloudy-day,"['KSMF', 'TS389', '72483023232', '72483993225', '72483323206', 'AN063', 'KSAC']",obs,NA,NA
2020-01-02,1577952000,17.0,4.2,9.1,17.0,2.5,8.5,5.0,78.1,...,16:56:06,1578012966,0.25,Partially cloudy,Partly cloudy throughout the day.,partly-cloudy-day,"['KSMF', 'TS388', 'TS389', '72483023232', '72483993225', 'AN063', '72483323206', 'KSAC', '72057600174']",obs,NA,NA
2020-01-03,1578038400,15.0,3.8,8.9,15.0,3.8,8.7,6.0,83.0,...,16:56:57,1578099417,0.28,Clear,Clear conditions throughout the day.,clear-day,"['KSMF', 'TS389', '72483023232', '72483993225', 'AN063', '72483323206', 'KSAC']",obs,NA,NA
2020-01-04,1578124800,13.6,5.7,9.3,13.6,2.8,8.5,7.6,89.6,...,16:57:48,1578185868,0.31,"Rain, Partially cloudy",Partly cloudy throughout the day with morning rain.,rain,"['KSMF', 'TS389', '72483023232', '72483993225', 'AN063', '72483323206', 'KSAC', '72057600174']",obs,NA,NA
2020-01-05,1578211200,15.1,3.2,8.4,15.1,1.2,7.8,4.9,80.9,...,16:58:41,1578272321,0.34,Partially cloudy,Clearing in the afternoon.,partly-cloudy-day,"['KSMF', 'TS389', '72483023232', '72483993225', '72483323206', 'AN063', 'KSAC']",obs,NA,NA
2020-01-06,1578297600,13.3,3.0,7.6,13.3,2.3,6.8,2.9,73.8,...,16:59:36,1578358776,0.37,Clear,Clear conditions throughout the day.,clear-day,"['KSMF', 'TS389', '72483023232', '72483993225', '72483323206', 'AN063', 'KSAC', '72057600174']",obs,NA,NA


### Subset Columns

**Column Selection**: we select a subset of columns, including `datetime`, `tempmax`, `tempmin`, `humidity`, `precip`, and `windspeed`. These columns are chosen based on their relevance to my analysis:

In [46]:
# Subset a data frame with the given column names:
Cali_2020 <- Cali_2020[ , c('datetime', 'tempmax', 'tempmin', 'humidity', 
                            'precip', 'windspeed')]
head(Cali_2020)

datetime,tempmax,tempmin,humidity,precip,windspeed
<date>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
2020-01-01,14.3,5.4,85.7,0.000,11.2
2020-01-02,17.0,4.2,78.1,0.000,19.0
2020-01-03,15.0,3.8,83.0,0.000,9.0
2020-01-04,13.6,5.7,89.6,1.736,16.7
2020-01-05,15.1,3.2,80.9,0.000,16.2
2020-01-06,13.3,3.0,73.8,0.000,18.7



Repeating the above cleaning procedure 5 more times on the rest of 5 datasets: 

In [47]:
#| code-fold: true
Cali_2017 <- read_csv("../data/raw-data/Cali_2017.csv", show_col_types = FALSE)
Cali_2017 <- Cali_2017[ , c('datetime', 'tempmax', 'tempmin', 'humidity', 
                            'precip', 'windspeed')]

Cali_2015 <- read_csv("../data/raw-data/Cali_2015.csv", show_col_types = FALSE)
Cali_2015 <- Cali_2015[ , c('datetime', 'tempmax', 'tempmin', 'humidity', 
                            'precip', 'windspeed')]

Cali_2014 <- read_csv("../data/raw-data/Cali_2014.csv", show_col_types = FALSE)
Cali_2014 <- Cali_2014[ , c('datetime', 'tempmax', 'tempmin', 'humidity', 
                            'precip', 'windspeed')]

Cali_2010 <- read_csv("../data/raw-data/Cali_2010.csv", show_col_types = FALSE)
Cali_2010 <- Cali_2010[ , c('datetime', 'tempmax', 'tempmin', 'humidity', 
                            'precip', 'windspeed')]


### Merging Datasets

**Data Integration**: we perform a left merge on the `Cali_2020` datasets, merging them with the `Cali_20_Wildfire` dataset (wildfire data) based on the `datetime` and `StartDate` columns. The resulting dataset, "Cali20_climate_fire," combines climate and wildfire data for the year 2020 in the state of California:

In [48]:
# Left merge the two datasets based on the 'datetime' and 'StartDate' columns
Cali20_climate_fire <- merge(Cali_2020, Cali_20_Wildfire, by.x = "datetime", by.y = "StartDate", all.x = TRUE)
head(Cali20_climate_fire)

,datetime,tempmax,tempmin,humidity,precip,windspeed,AcresBurned,Cause,StructureDam,Fatalities
,<date>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>
1,2020-01-01,14.3,5.4,85.7,0.000,11.2,NA,NA,NA,NA
2,2020-01-02,17.0,4.2,78.1,0.000,19.0,NA,NA,NA,NA
3,2020-01-03,15.0,3.8,83.0,0.000,9.0,NA,NA,NA,NA
4,2020-01-04,13.6,5.7,89.6,1.736,16.7,NA,NA,NA,NA
5,2020-01-05,15.1,3.2,80.9,0.000,16.2,NA,NA,NA,NA
6,2020-01-06,13.3,3.0,73.8,0.000,18.7,NA,NA,NA,NA



### Create New Column

In the merged dataset, `Cali20_climate_fire`, we create a new column called `fire`. This column represents the presence of a wildfire based on the "AcresBurned" column. If there is a value in "AcresBurned," the "fire" column is marked as "Yes"; otherwise, it's marked as "No."

In [49]:
# Create a new column 'fire' based on the 'AcresBurned' column
Cali20_climate_fire$fire <- ifelse(!is.na(Cali20_climate_fire$AcresBurned), "Yes", "No")
head(Cali20_climate_fire)

,datetime,tempmax,tempmin,humidity,precip,windspeed,AcresBurned,Cause,StructureDam,Fatalities,fire
,<date>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<chr>
1,2020-01-01,14.3,5.4,85.7,0.000,11.2,NA,NA,NA,NA,No
2,2020-01-02,17.0,4.2,78.1,0.000,19.0,NA,NA,NA,NA,No
3,2020-01-03,15.0,3.8,83.0,0.000,9.0,NA,NA,NA,NA,No
4,2020-01-04,13.6,5.7,89.6,1.736,16.7,NA,NA,NA,NA,No
5,2020-01-05,15.1,3.2,80.9,0.000,16.2,NA,NA,NA,NA,No
6,2020-01-06,13.3,3.0,73.8,0.000,18.7,NA,NA,NA,NA,No


In [50]:
#| code-fold: true

# Left merge the two datasets based on the 'datetime' and 'StartDate' columns
Cali17_climate_fire <- merge(Cali_2017, Cali_17_Wildfire, by.x = "datetime", by.y = "StartDate", all.x = TRUE)

# Create a new column 'fire' based on the 'AcresBurned' column
Cali17_climate_fire$fire <- ifelse(!is.na(Cali17_climate_fire$AcresBurned), "Yes", "No")

# Left merge the two datasets based on the 'datetime' and 'StartDate' columns
Cali15_climate_fire <- merge(Cali_2015, Cali_15_Wildfire, by.x = "datetime", by.y = "StartDate", all.x = TRUE)

# Create a new column 'fire' based on the 'AcresBurned' column
Cali15_climate_fire$fire <- ifelse(!is.na(Cali15_climate_fire$AcresBurned), "Yes", "No")

# Left merge the two datasets based on the 'datetime' and 'StartDate' columns
Cali14_climate_fire <- merge(Cali_2014, Cali_14_Wildfire, by.x = "datetime", by.y = "StartDate", all.x = TRUE)

# Create a new column 'fire' based on the 'AcresBurned' column
Cali14_climate_fire$fire <- ifelse(!is.na(Cali14_climate_fire$AcresBurned), "Yes", "No")

# Left merge the two datasets based on the 'datetime' and 'StartDate' columns
Cali10_climate_fire <- merge(Cali_2010, Cali_10_Wildfire, by.x = "datetime", by.y = "StartDate", all.x = TRUE)

# Create a new column 'fire' based on the 'AcresBurned' column
Cali10_climate_fire$fire <- ifelse(!is.na(Cali10_climate_fire$AcresBurned), "Yes", "No")

Now, let's combine the California wildfire statistics and weather statistics into a single dataset: 

In [53]:
# List of data file names
data_files <- c("../data/cleaned-data/Cali20_climate_fire.csv", "../data/cleaned-data/Cali17_climate_fire.csv", "../data/cleaned-data/Cali15_climate_fire.csv", "../data/cleaned-data/Cali14_climate_fire.csv", "../data/cleaned-data/Cali10_climate_fire.csv")

# Read and combine the datasets
combined_data <- bind_rows(lapply(data_files, read.csv))

# Save the combined dataset to a new CSV file
write.csv(combined_data, "../data/cleaned-data/CA_climate_fire.csv", row.names = FALSE)


## Text Data on Wildfire Prevention

In this section, I will outline the data cleaning process for text data obtained from a JSON file containing articles. The objective is to prepare the text data for further analysis on the attitudes of the public towards wildfire prevention, as well as create a clean, structured dataset. The code shown below processes the data, cleans it, and enriches it with sentiment analysis.

### Data Loading

The first step is to load the raw text data from the `articles.json` file. The data consists of various attributes for each article, including `source`, `author`, `title`, `description`, `content`, and `publishedAt`:


In [2]:
#| echo: false
import requests
import json
import re
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer


In [4]:
# Load the JSON data from the file
with open("../data/raw-data/articles.json", "r") as json_file:
    data = json.load(json_file)
    articles = pd.DataFrame(data)
print(articles)

                                               source  \
0              {'id': 'bbc-news', 'name': 'BBC News'}   
1                     {'name': 'Scientific American'}   
2                                     {'name': 'Vox'}   
3                                  {'name': 'Forbes'}   
4   {'id': 'al-jazeera-english', 'name': 'Al Jazee...   
..                                                ...   
95                     {'name': 'Marketscreener.com'}   
96                          {'name': 'GlobeNewswire'}   
97                          {'name': 'GlobeNewswire'}   
98                     {'name': 'Marketscreener.com'}   
99                              {'name': '[Removed]'}   

                                               author  \
0                    https://www.facebook.com/bbcnews   
1                                      Curtis Abraham   
2                                          Kylie Mohr   
3   Jamie Hailstone, Contributor, \n Jamie Hailsto...   
4                             

### Data Cleaning Functions

To ensure data quality, several data cleaning functions are used:

- **String Cleaning**: A custom function, `string_cleaner`, is applied to clean and normalize text data. This function handles punctuation, whitespace, character replacements, and converts text to lowercase:


In [5]:
verbose=True

def string_cleaner(input_string):
    out = ''
    try: 
        out=re.sub(r"""
                    [,.;@#?!&$-]+  # Accept one or more copies of punctuation
                    \ *           # plus zero or more copies of a space,
                    """,
                    " ",          # and replace it with a single space
                    input_string, flags=re.VERBOSE)

        #REPLACE SELECT CHARACTERS WITH NOTHING
        out = re.sub('[’.]+', '', input_string)

        #ELIMINATE DUPLICATE WHITESPACES USING WILDCARDS
        out = re.sub(r'\s+', ' ', out)

        #CONVERT TO LOWER CASE
        out=out.lower()
    except:
        print("ERROR")
        out=''
    return out

### Cleaning Process

The cleaning process proceeds as follows:

- **Column Selection**: A subset of relevant columns is selected for further processing, including "source," "author," "title," "description," "content," and "publishedAt."

- **Author and Source Handling**: An error check is performed to ensure that the "author" is not mistakenly the same as the "source."

- **Date Validation**: A regular expression is used to validate the date format in the "publishedAt" column.

In [6]:
article_keys = articles.columns
print("AVAILABLE KEYS:")
print(article_keys)

index = 0
cleaned_data = []

for _, article in articles.iterrows():
    tmp = []
    if verbose:
        print("#------------------------------------------")
        print("#", index)
        print("#------------------------------------------")

    for key in article_keys:
        if verbose:
            print("----------------")
            print(key)
            print(article[key])  # Access the value directly

        if key == 'source':
            src = string_cleaner(article['source']['name'])
            tmp.append(src)

        if key == 'author':
            author = string_cleaner(article['author'])
            # ERROR CHECK (SOMETIMES AUTHOR IS SAME AS PUBLICATION)
            if src in author:
                print(" AUTHOR ERROR:", author)
                author = 'NA'
            tmp.append(author)

        if key == 'title':
            tmp.append(string_cleaner(article['title']))

        if key == 'description':
            tmp.append(string_cleaner(article['description']))

        if key == 'content':
            tmp.append(string_cleaner(article['content']))

        if key == 'publishedAt':
            # DEFINE DATA PATTERN FOR RE TO CHECK  .* --> wildcard
            ref = re.compile('.*-.*-.*T.*:.*:.*Z')
            date = article['publishedAt']
            if not ref.match(date):
                print(" DATE ERROR:", date)
                date = "NA"
            tmp.append(date)

    cleaned_data.append(tmp)
    index += 1

AVAILABLE KEYS:
Index(['source', 'author', 'title', 'description', 'url', 'urlToImage',
       'publishedAt', 'content'],
      dtype='object')
#------------------------------------------
# 0
#------------------------------------------
----------------
source
{'id': 'bbc-news', 'name': 'BBC News'}
----------------
author
https://www.facebook.com/bbcnews
----------------
title
Voice referendum: Lies fuel racism ahead of Australia's Indigenous vote
----------------
description
Experts express concerns for First Nations people as a referendum debate turns increasingly divisive.
----------------
url
https://www.bbc.co.uk/news/world-australia-66470376
----------------
urlToImage
https://ichef.bbci.co.uk/news/1024/branded_news/13D39/production/_130790218_gettyimages-1593394059-1.jpg
----------------
publishedAt
2023-09-05T21:23:49Z
----------------
content
"People have been let off the leash," Thomas Mayo says quietly, swiping through screenshots. 
Racist memes depicting First Nations Austra

- **Filtering Removed Data**: Rows containing "[removed]" in the "content" column are removed.

In [7]:
# Define a filter condition to check for "[removed]" in column 5
filter_condition = lambda article: article[5] != "[removed]"
# Use a list comprehension to create a new cleaned_data list with rows that satisfy the condition
cleaned_data_filtered = [article for article in cleaned_data if filter_condition(article)]

- **Count Vectorization**: The article contents are transformed using CountVectorizer to create a matrix of token counts, allowing text data to be represented in a structured format.

In [8]:
def apply_count_vectorizer(text_data):
    # Create an instance of CountVectorizer
    vectorizer = CountVectorizer()
    
    # Fit and transform the text data
    X = vectorizer.fit_transform(text_data)
    
    # Get the feature names (tokens)
    feature_names = vectorizer.get_feature_names_out()
    
    # Convert the matrix to a DataFrame for better readability
    count_matrix = pd.DataFrame(X.toarray(), columns=feature_names)
    
    return count_matrix
# Apply CountVectorizer to article contents
content_data = [article[5] for article in cleaned_data_filtered] 
content_count_matrix = apply_count_vectorizer(content_data)
print(content_count_matrix)

    000  007  01  015  07  08  09  100  10311  10521  ...  wrapped  yea  year  \
0     0    0   0    0   0   0   0    0      0      0  ...        0    0     0   
1     0    0   0    0   0   0   0    0      0      0  ...        0    0     0   
2     0    0   0    0   0   0   0    0      0      0  ...        0    0     0   
3     0    0   0    0   0   0   0    0      0      0  ...        0    0     0   
4     0    0   0    0   0   0   0    0      0      0  ...        0    0     0   
..  ...  ...  ..  ...  ..  ..  ..  ...    ...    ...  ...      ...  ...   ...   
93    0    0   0    0   0   0   0    0      0      0  ...        0    0     0   
94    0    0   0    0   0   0   0    0      0      0  ...        0    0     0   
95    0    0   0    0   0   0   1    0      0      0  ...        0    0     0   
96    0    0   0    0   0   0   0    0      0      0  ...        0    0     0   
97    0    0   0    0   0   0   0    0      0      0  ...        0    0     0   

    years  york  you  young

- **Sentiment Analysis**: The sentiment of each article's content is analyzed using the SentimentIntensityAnalyzer from the NLTK library. The resulting sentiment scores are stored for each article:

In [11]:
from nltk.sentiment import SentimentIntensityAnalyzer 

# Initialize the SentimentIntensityAnalyzer
sia = SentimentIntensityAnalyzer()

# Analyze the sentiment of each article's content and store the sentiment scores
sentiments = []

for content in content_data:
    sentiment_score = sia.polarity_scores(content)
    sentiments.append(sentiment_score)

# Create a DataFrame for the sentiment scores
sentiments_df = pd.DataFrame(sentiments)

In [12]:
print(sentiments_df)

      neg    neu    pos  compound
0   0.121  0.879  0.000   -0.6124
1   0.154  0.846  0.000   -0.7184
2   0.000  0.946  0.054    0.2263
3   0.074  0.926  0.000   -0.3400
4   0.000  0.917  0.083    0.4588
..    ...    ...    ...       ...
93  0.000  0.927  0.073    0.2960
94  0.060  0.940  0.000   -0.2263
95  0.058  0.942  0.000   -0.2263
96  0.058  0.942  0.000   -0.2263
97  0.060  0.940  0.000   -0.2263

[98 rows x 4 columns]


- **Combining Dataframes**: The cleaned text data is combined with the sentiment scores, resulting in a DataFrame that includes sentiment attributes.

- **Renaming Columns**: The columns of the combined DataFrame are renamed for clarity and consistency.

In [13]:
cleaned_text_data = pd.read_csv("../data/cleaned-data/cleaned_text_data.csv")
# Combine the sentiment scores with the original article DataFrame
articles_with_sentiment = pd.concat([cleaned_text_data, sentiments_df], axis=1)

# Rename columns
articles_with_sentiment = articles_with_sentiment.rename(columns={
    '...1': 'index',
    '0': 'source',
    '1': 'author',
    '2': 'title',
    '3': 'description',
    '4': 'date',
    '5': 'content'
})

# Print the DataFrame to verify the changes
print(articles_with_sentiment)

                 source                          author  \
0              bbc news  https://wwwfacebookcom/bbcnews   
1   scientific american                  curtis abraham   
2                   vox                      kylie mohr   
3                forbes                             NaN   
4    al jazeera english            matthaios tsimitakis   
..                  ...                             ...   
93    marketscreenercom                      prnewswire   
94    marketscreenercom                      prnewswire   
95        globenewswire                   pomerantz llp   
96        globenewswire                   pomerantz llp   
97    marketscreenercom                      prnewswire   

                                                title  \
0   voice referendum: lies fuel racism ahead of au...   
1   large herbivores can help prevent massive wild...   
2             wildfire risk is everywhere and growing   
3   why now is the time to focus on wildfire preve...   
4     